# Проверка написан ли текст ИИ

In [1]:
import getpass
import os

if "GIGACHAT_CREDENTIALS" not in os.environ:
    os.environ["GIGACHAT_CREDENTIALS"] = getpass.getpass("Credentials от GigaChat")

In [3]:
from gigachat import GigaChat

client = GigaChat(
    scope="GIGACHAT_API_CORP",
    verify_ssl_certs=False
)

In [4]:
text = """Котики — это милые и пушистые домашние животные, которые относятся к семейству кошачьих. 
Они известны своим дружелюбным характером, игривостью и мягкой шерстью. 
Котики могут быть разных пород, каждая из которых имеет свои уникальные особенности. 
А этот текст писала точно не нейросеть, 
потому что это пишу я. У меня особо нечего сказать про котов,
кроме того, что они мягкие и пушистые, мурчащие существа.
Эти животные стали популярными благодаря своей способности поднимать настроение людям 
и создавать уютную атмосферу в доме."""
ai_result = client.check_ai(text, "GigaCheckDetection")
ai_result

AICheckResult(category='mixed', characters=533, tokens=224, ai_intervals=[[1, 262], [415, 533]])

In [7]:
from IPython.display import display, HTML

def ai_intervals_to_html(text, ai_intervals):
    """
    Функция получает строку `text` и список интервалов в виде [(start, end), ...],
    где части текста, попадающие в интервалы, будут выделены красным цветом, а остальные – чёрным.
    Если список интервалов пустой, возвращается исходный текст.
    """
    # Если интервалов нет, просто возвращаем текст, обернутый в span с чёрным цветом.
    if not ai_intervals:
        return f"<span style='color: black'>{text}</span>"
    
    output = ""
    # Если до первого интервала есть текст, добавляем его чёрным цветом.
    if ai_intervals[0][0] > 0:
        output += f"<span style='color: black'>{text[:ai_intervals[0][0]]}</span>"
        
    for i, (start, end) in enumerate(ai_intervals):
        # Если между интервалами есть промежуток > 1 символа, добавляем его чёрным цветом.
        if i > 0 and (start - ai_intervals[i-1][1] > 1):
            output += f"<span style='color: black'>{text[ai_intervals[i-1][1]:start]}</span>"
        # Добавляем выделенный красным цветом фрагмент.
        output += f"<span style='color: red'>{text[start:end]}</span>"
    
    # Если после последнего интервала остаётся текст, добавляем его чёрным цветом.
    if ai_intervals[-1][1] < len(text):
        output += f"<span style='color: black'>{text[ai_intervals[-1][1]:]}</span>"
    
    return output.replace("\n", "<br/>")

html = ai_intervals_to_html(text, ai_result.ai_intervals)
display(HTML(f"<div style='background:white'>{html}</div>"))
